In [1]:
! pip install datasets transformers rouge-score nltk contractions torchmetrics scipy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 25.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=dfe831df27bf828de427126a97a1eefaef29a91fec2f453718495134e75d7667
  Stored in directory: /root/.cache/pip/wheels/8e/6b/70/59daa7c90a238610e34bac5916e001fe3d9bb0ec59c8cf5518
Successfully built rouge-score


In [2]:
import gc
import time
import re
import argparse
import warnings
import os

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchmetrics

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import contractions

from tqdm.auto import tqdm

import transformers
import logging


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [3]:
# Basic Preprocessing
def expand_contractions(sentence):
  contractions_expanded = [contractions.fix(word) for word in sentence.split()]
  return ' '.join(contractions_expanded)

def lower_case(sentence):
  return ' '.join([word.lower() for word in sentence.split()])

def remove_punctuation(sentence):
  return ' '.join([re.sub(r'[^\w\s]', '', word) for word in sentence.split()])

def preprocess(lst, process=True,):
  if process == True:
    for i, sent in enumerate(lst):
      lst[i] = lower_case(remove_punctuation(expand_contractions(sent)))
      
  
  return lst

In [4]:
# Importing only the data for which we have summaries
data_with_summaries = pd.read_csv('/kaggle/input/tweetsummemnlp2017/9_tweet_grouped_data_no_dups.csv').iloc[:3612]

In [5]:
# Importing the summaries
summaries = pd.read_csv('/kaggle/input/tweetsummemnlp2017/ChatGPT-Summaries.csv', sep=';')

In [6]:
# Preprocesiing the summaries
start_time = time.time()
summaries['generated_summary'] = preprocess(list(summaries['generated_summary']))
print('Time taken for preprocessing', time.time() - start_time)

Time taken for preprocessing 0.10208368301391602


### Creating the Dataset

In [7]:
class SummaryTrainDataset(Dataset):
    def __init__(self, tokenizer, tweet_data, summary_data, max_len=256):

        self.tweet_column = "tweet"
        self.group_id_column = "group_id"
        self.summary_column = "generated_summary"
        self.tweet_data = tweet_data
        self.summary_data = summary_data
        self.max_len = max_len
        self.tokenizer = tokenizer
        
        self.inputs = []
        self.targets = []
        self.input_texts = []
        self.summary_texts = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        ip = self.inputs[index]
        tgt = self.targets[index]
        ip_text = self.input_texts[index]
        tgt_text = self.summary_texts[index]
        
        
        return {"input": ip, "target": tgt, "input_text": ip_text, "target_text": tgt_text}


    def _build(self):
        for idx in tqdm(range(len(self.summary_data))):
            summ = self.summary_data.iloc[idx][self.summary_column]
            tweet_subset = self.tweet_data[self.tweet_data[self.group_id_column] == self.summary_data.iloc[idx][self.group_id_column]]['tweet'].iloc[:8]
            ip = '. '.join(list(tweet_subset))
            
            del tweet_subset
            gc.collect()
            
            self.input_texts.append(ip)
            self.summary_texts.append(summ)
            
            ip = "summarize: " + ip
            
            tokenized_inputs = self.tokenizer(
                ip, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt'
            )
            
            tokenized_targets = self.tokenizer(
                text_target=summ, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt'
            )
            
            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [8]:
class SummaryTestDataset(Dataset):
    def __init__(self, tokenizer, tweet_data, summary_data, max_len=256):

        self.tweet_column = "tweet"
        self.group_id_column = "group_id"
        self.summary_column = "generated_summary"
        self.tweet_data = tweet_data
        self.summary_data = summary_data
        self.max_len = max_len
        self.tokenizer = tokenizer
        
        self.inputs = []
        self.targets = []
        self.input_texts = []
        self.summary_texts = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        ip = self.inputs[index]
        tgt = self.targets[index]
        ip_text = self.input_texts[index]
        tgt_text = self.summary_texts[index]
        
        return {"input": ip, "target": tgt, "input_text": ip_text, "target_text": tgt_text}


    def _build(self):
        for idx in tqdm(range(len(self.summary_data))):
            summ = self.summary_data.iloc[idx][self.summary_column]
            tweet_subset = self.tweet_data[self.tweet_data[self.group_id_column] == self.summary_data.iloc[idx][self.group_id_column]]['tweet'].iloc[8:]
            ip = '. '.join(list(tweet_subset))
            
            del tweet_subset
            gc.collect()
            
            self.input_texts.append(ip)
            self.summary_texts.append(summ)
            
            ip = "summarize: " + ip
            summ = summ
            
            tokenized_inputs = self.tokenizer(
                ip, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt'
            )
            
            tokenized_targets = self.tokenizer(
                text_target=summ, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt'
            )
            
            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [9]:
model_checkpoint = "t5-base"

In [10]:
from transformers import T5TokenizerFast

tokenizer = T5TokenizerFast.from_pretrained(
    model_checkpoint,
    model_max_length = 256,
)

In [11]:
train_dataset = SummaryTrainDataset(tokenizer, data_with_summaries, summaries)
test_dataset = SummaryTestDataset(tokenizer, data_with_summaries, summaries)

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

## Models

In [13]:
# Discriminator Model: Bert as a Sequence Classifier
from transformers import BertForSequenceClassification, BertTokenizerFast

discriminator_checkpoint = 'bert-base-uncased'
discriminator_tokenizer = BertTokenizerFast.from_pretrained(
    discriminator_checkpoint,
)
discriminator = BertForSequenceClassification.from_pretrained(discriminator_checkpoint,
                                                              num_labels = 1,
                                                             ).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
# Generator Model: T5 as a Summarizer
from transformers import T5ForConditionalGeneration

generator_checkpoint = 't5-base'
generator = T5ForConditionalGeneration.from_pretrained(generator_checkpoint, max_length = 256).to(device)

## Loss and Optimisers

In [15]:
# Binary Cross Entropy loss for Discriminator
discriminator_loss_fn = nn.BCELoss()
# Generator Loss received as an output of T5

gen_lr = 5e-5
disc_lr = 2e-4
generator_optimiser = torch.optim.AdamW(generator.parameters(), lr = gen_lr, betas=(0.8, 0.9))
discriminator_optimiser = torch.optim.AdamW(discriminator.parameters(), lr = disc_lr, betas=(0.9, 0.999))

## Metrics

In [16]:
# ROUGE and BERT
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics.text.bert import BERTScore

In [17]:
# Evaluator Model for Semantic Similarity: Bert Large
from transformers import BertModel, BertTokenizerFast

evaluator_checkpoint = 'bert-large-uncased'
evaluator = BertModel.from_pretrained(evaluator_checkpoint).to(device)
evaluator_tokenizer = BertTokenizerFast.from_pretrained(
    evaluator_checkpoint,
)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Training Loop
def train(
    generator,
    discriminator,
    discriminator_tokenizer,
    generator_tokenizer,
    generator_optimiser,
    discriminator_optimiser,
    train_loader,
    epoch
) -> None:
    
    generator.train()
    discriminator.train()
    start_time = time.time()
    log_interval = 5
    
    for i, batch in enumerate(train_loader):
        # DISCRIMINATOR TRAIN STEP
        # discriminator training on real summary
        discriminator_optimiser.zero_grad()
        full_input = batch['target_text']
        tokenised = discriminator_tokenizer(text = full_input,
                                        max_length=256,
                                        padding='max_length',
                                        return_tensors='pt',
                                        truncation=True
                                       )
        target = torch.ones((len(batch['target_text']), 1), dtype=torch.int32).type(torch.LongTensor).to(device)
        logits = discriminator(**tokenised.to(device),
                               labels = target
                              ).logits
        prob = torch.sigmoid(logits)
        loss = discriminator_loss_fn(prob.to('cpu'),
                                     torch.ones((len(batch['target_text']), 1)))
        loss.backward()
        discriminator_optimiser.step()
        
        del full_input, tokenised, logits, prob, loss
        
        # discriminator training on generated summary
        discriminator_optimiser.zero_grad()
        gen_sum = generator.generate(
            inputs = batch['input']['input_ids'].squeeze(1).detach().to(device),
            num_beams = 4,
            top_k = 20,
            max_length = 256,
            early_stopping = True,
            num_return_sequences = 1
        )
        gen_sum = generator_tokenizer.batch_decode(
            gen_sum,
            skip_special_tokens=False,
            clean_up_tokenization_spaces=True
        )
        full_input = gen_sum
        tokenised = discriminator_tokenizer(text = full_input,
                                        max_length=256,
                                        padding='max_length',
                                        return_tensors='pt',
                                        truncation=True
                                       )
        target = torch.ones((len(batch['target_text']), 1), dtype=torch.int32).type(torch.LongTensor).to(device)
        logits = discriminator(**tokenised.to(device),
                               labels = target
                              ).logits
        prob = torch.sigmoid(logits)
        loss = discriminator_loss_fn(prob.to('cpu'),
                                     torch.zeros((len(batch['target_text']), 1)))
        loss.backward()
        discriminator_optimiser.step()
        
        del full_input, tokenised, logits, prob, loss, gen_sum
        
        # ADVERSARIAL TRAINING FOR 
        # generator training
        generator_optimiser.zero_grad()
        gen_sum = generator.generate(
            inputs = batch['input']['input_ids'].squeeze(1).detach().to(device),
            num_beams = 4,
            top_k = 20,
            max_length = 256,
            early_stopping = True,
            num_return_sequences = 1
        )
        gen_sum = generator_tokenizer.batch_decode(
            gen_sum,
            skip_special_tokens=False,
            clean_up_tokenization_spaces=True
        )
        full_input = gen_sum
        tokenised = discriminator_tokenizer(text = full_input,
                                        max_length=256,
                                        padding='max_length',
                                        return_tensors='pt',
                                        truncation=True
                                       )
        target = torch.ones((len(batch['target_text']), 1), dtype=torch.int32).type(torch.LongTensor).to(device)
        logits = discriminator(**tokenised.to(device),
                               labels = target
                              ).logits
        prob = torch.sigmoid(logits)
        adv_loss = discriminator_loss_fn(prob.to('cpu'),
                                     torch.zeros((len(batch['target_text']), 1)))
        
        seq2seqloss = generator(
            input_ids = batch['input']['input_ids'].squeeze(1).to(device),
            attention_mask = batch['input']['attention_mask'].squeeze(1).to(device),
            labels = batch['target']['input_ids'].squeeze(1).to(device)
        ).loss
        
        loss = seq2seqloss + adv_loss
        
        loss.backward()
        generator_optimiser.step()
        
        del full_input, tokenised, logits, prob, loss, gen_sum
        
        if (i+1) % log_interval == 0:
            # comment if no gpu
            torch.cuda.empty_cache()
            gc.collect()
            
            s_per_batch = (time.time() - start_time) / log_interval
            print(f'| epoch {epoch:3d} | {i + 1: 5d} batch | '
                 f'sec/batch {s_per_batch:5.2f} | ')
            start_time = time.time()
            
#         # TURN OFF TO RESUME NORMAL OPERATION
#         break
            
def evaluate(
    model,
    model_tokenizer,
    data_loader
):
    model.eval()
    
    # Initialise metrics
    rouge = ROUGEScore()
    bert_score = BERTScore(
        model_name_or_path = evaluator_checkpoint,
        model = evaluator.to(device),
        device = device
    )

    rogue = rouge.to(device)
    bert_score = bert_score.to(device)
    
    final_bert_score = []
    
    start_time = time.time()
    generation_time = 0
    rouge_time = 0
    bert_score_time = 0
    
    log_interval = 5
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with torch.no_grad():
            for i, batch in enumerate(data_loader):
                gen_time = time.time()
                output = model.generate(
                    input_ids = batch['input']['input_ids'].squeeze(1).to(device),
                    attention_mask = batch['input']['attention_mask'].squeeze(1).to(device),
                )
                output = model_tokenizer.batch_decode(
                    output,
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=True
                )
                generation_time += time.time() - gen_time

                score_time = time.time()
                rouge.update(output, batch['target_text'])
                rouge_time += time.time() - score_time

                score_time = time.time()
                score = bert_score(output, batch['target_text'])
                for scr in score['f1']:
                    final_bert_score.append(scr)

                bert_score_time += time.time() - score_time
                
                if (i+1) % log_interval == 0:
                    s_per_batch = (time.time() - start_time) / log_interval
                    print(f'| {i + 1: 5d} batch | '
                         f'sec/batch {s_per_batch:5.2f} | ')
                    start_time = time.time()

#                 # TURN OFF TO RESUME NORMAL OPERATION
#                 break

        final_rouge = rouge.compute()
        final_bert_score = np.mean(np.array(final_bert_score))


        return final_rouge, final_bert_score
            

In [20]:
def model_trainer(
    generator,
    discriminator,
    discriminator_tokenizer,
    generator_tokenizer,
    generator_optimiser,
    discriminator_optimiser,
    train_loader,
    test_loader,
    epochs
):
    
    for epoch in tqdm(range(epochs)):
        epoch_start_time = time.time()
        
        train(
            generator,
            discriminator,
            discriminator_tokenizer,
            generator_tokenizer,
            generator_optimiser,
            discriminator_optimiser,
            train_loader,
            epoch
        )
        print("-----EVALUATION STARTED-----")
        rouge, bert_score = evaluate(generator, generator_tokenizer, test_loader)
        
        elapsed = time.time() - epoch_start_time
        
        print('-' * 100)
        print(
            f'| end of epoch {epoch + 1:3d} | time: {elapsed:5.2f}s | '
            f'Rouge1 {rouge["rouge1_fmeasure"].item():4.4f} | Rouge2 {rouge["rouge2_fmeasure"].item():4.4f} | '
            f'RougeL {rouge["rougeL_fmeasure"].item():4.4f} | RougeL Sum {rouge["rougeLsum_fmeasure"].item():4.4f} | '
            f'BERT Score {float(bert_score.squeeze()):4.4f}'
             )
        print('-' * 100)
        
        generator.save_pretrained('/kaggle/working/model' + str(epoch), from_pt=True)

In [21]:
# Make the Train and Test Loader
train_loader = DataLoader(dataset = train_dataset, batch_size = 4, drop_last=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 16, drop_last=True)

In [22]:
# Train the model
n_epochs = 7

model_trainer(
    generator,
    discriminator,
    discriminator_tokenizer,
    tokenizer,
    generator_optimiser,
    discriminator_optimiser,
    train_loader,
    test_loader,
    epochs = n_epochs
)

  0%|          | 0/7 [00:00<?, ?it/s]

| epoch   0 |     5 batch | sec/batch  3.66 | 
| epoch   0 |    10 batch | sec/batch  3.74 | 
| epoch   0 |    15 batch | sec/batch  4.12 | 
| epoch   0 |    20 batch | sec/batch  4.48 | 
| epoch   0 |    25 batch | sec/batch  4.45 | 
| epoch   0 |    30 batch | sec/batch  4.63 | 
| epoch   0 |    35 batch | sec/batch  4.82 | 
| epoch   0 |    40 batch | sec/batch  6.18 | 
| epoch   0 |    45 batch | sec/batch  6.00 | 
| epoch   0 |    50 batch | sec/batch  6.90 | 
| epoch   0 |    55 batch | sec/batch  8.58 | 
| epoch   0 |    60 batch | sec/batch  8.31 | 
| epoch   0 |    65 batch | sec/batch  9.36 | 
| epoch   0 |    70 batch | sec/batch  9.49 | 
| epoch   0 |    75 batch | sec/batch  8.24 | 
-----EVALUATION STARTED-----
|     5 batch | sec/batch  5.16 | 
|    10 batch | sec/batch  4.96 | 
|    15 batch | sec/batch  4.94 | 
----------------------------------------------------------------------------------------------------
| end of epoch   1 | time: 554.94s | Rouge1 0.0063 | Rouge2 

In [36]:
generator.save_pretrained('/kaggle/working/generator_final', from_pt=True)
discriminator.save_pretrained('/kaggle/working/discriminator_final', from_pt=True)

In [35]:
torch.cuda.empty_cache()
gc.collect()

3730

## Results and Examples from here on

In [23]:
def get_discriminator(batch):
    discriminator.eval()
    with torch.no_grad():
        full_input = batch['target_text']
        tokenised = discriminator_tokenizer(text = full_input,
                                        max_length=256,
                                        padding='max_length',
                                        return_tensors='pt',
                                        truncation=True
                                       )
        logits1 = discriminator(**tokenised.to(device),
#                                        labels = target
                                      ).logits
        
        gen_sum = generator.generate(
            inputs = batch['input']['input_ids'].squeeze(1).detach().to(device),
            num_beams = 4,
            top_k = 20,
            max_length = 256,
            early_stopping = True,
            num_return_sequences = 1
        )
        gen_sum = tokenizer.batch_decode(
            gen_sum,
            skip_special_tokens=False,
            clean_up_tokenization_spaces=True
        )
#         full_input = [batch['input_text'][i] + gen_sum[i] for i in range(len(batch['target_text']))]
        full_input = gen_sum
        tokenised = discriminator_tokenizer(text = full_input,
                                        max_length=256,
                                        padding='max_length',
                                        return_tensors='pt',
                                        truncation=True
                                       )
        logits2 = discriminator(**tokenised.to(device),
#                                        labels = target
                                      ).logits
        
        return logits1, logits2

In [24]:
def get_generated_summary(batch):
    generator.eval()
    with torch.no_grad():
        output = generator.generate(
            input_ids = batch['input']['input_ids'].squeeze(1).to(device),
            attention_mask = batch['input']['attention_mask'].squeeze(1).to(device),
    #                     num_beams = 4,
    #                     top_k = 20,
    #                     max_length = 256,
    #                     early_stopping = True,
    #                     num_return_sequences = 1
        )
        output = tokenizer.batch_decode(
            output,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )

        return output

After lr finetune, 7 epochs

In [25]:
i = 1
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

excellent executive decision trump wants to cancel air force one order from boeing. build it in mexico it will be cheaper that way trump wants to cancel air force one order from boeing. when did republicans become so fond of regulation trump wants 2cancel air force one order from boeing. trump lies again says jet costs more than 4 billion amp wants to cancel air force one order from boeing. trump lies again says jet costs more than 4 billion amp wants to cancel air force one order from boeing. presidentelect realdonaldtrump wants to cancel a government order for a new air force one. amazing leadership and not even sworn in trump wants to cancel air force one order from boeing. trump wants to cancel air force one order from boeing what about the money for security at trump tower



/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1292: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['the paragraphs report on presidentelect trumps decision to cancel an order for a new air force one from boeing which is a good executive decision and a good example of the importance of executive discretion in the government some articles mention the lack of funding for security at trump towers and the lack of transparency in the government the tone is critical of trumps decision and the potential impact on the economy']

donald trump wants to cancel the order for a new air force one from boeing claiming it is too expensive over 4 billion and suggesting building it in mexico would be cheaper he also suggests using his own jet and charging taxpayers there are criticisms of his decision including concerns about security at trump tower and the accuracy of his cost estimates
(tensor([[10.2265]], device='cuda:0'), tensor([[-10.1844]], device='cuda:0'))


In [26]:
i = 25
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

remarkable photos from nytimes the year in pictures 2016. this is excellent the year in pictures 2016. a year to be confounded shocked nytimes year in pictures 2016. some extraordinary photos here the nytimes year in pictures 2016. the year in pictures 2016 new york times. a treasure here gt gt the year in pictures 2016. a riveting curation the year in pictures 2016. what a year 2016 even tragedy can be captured beautifully the year in pictures 2016

['the new york times has published a collection of extraordinary photos highlighting the year in pictures 2016 the collection includes some of the best photos from the year even tragedy can be captured beautifully the collection includes some of the best photos from the year even though some have been criticized for their coverage the collection is a treasure trove of extraordinary photos from the new york times and other sources']

the new york times the year in pictures 2016 is a collection of remarkable and extraordinary photographs tha

In [27]:
i = 4
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

the new york times how to pounce on best credit card offers. how to pounce on best credit card offers before banks pull them. how to search for the best credit card deals before the disappear. how to pounce on best credit card offers last chance for freemoney just pick it up. 1500 in your pocket ronlieber explains how to pounce on the best credit card offers. how to pounce on best credit card offers por ron lieber en nyt. how to pounce on best credit card offers by ron lieber the new york times. new york times most viewed stories how to pounce on best credit card offers

['the new york times has compiled a list of tips on how to pounce on credit card offers before they disappear the list includes tips on how to search for the best credit card deals before banks pull them some articles have been viewed by thousands of people including ron lieber some articles have been criticized for not pinning on the best credit card offers while others have criticized the lack of transparency in the 

In [28]:
i = 9
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

38 dead and 155 wounded in twin bomb attack near istanbul stadium who is really behind this atta. 29 dead and 166 wounded in twin bomb attack near istanbul stadium. terror in istanbul 29 dead 166 wounded in twin bomb attack in turkey. thoughts and prayers to the victims 38 dead and 155 wounded in twin bomb attack near istanbul stadium foxnews. 15 dead 69 wounded in twin bomb attack in istanbul. thoughts and prayers to the victims 38 dead and 155 wounded in twin bomb attack near istanbul stadium foxnews. 38 dead and 155 wounded in twin bomb attack near istanbul stadium what goes around comes around. 38 dead and 155 wounded in twin bomb attack near istanbul stadium what goes around comes around

['the paragraphs report on a twin bomb attack near an istanbul stadium that left 38 people dead and 155 wounded the focus is on the attack and the potential for further violence in turkey with coverage highlighting the reporting by fox news and other sources']

a twin bomb attack near an istanbul

In [29]:
i = 15
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

at least 27 people killed and 57 others wounded when blasts went off near car spare parts shops in al sinak. two blasts in central baghdad kill at least 28. double bomb attack hits al sinak market at least 21 killed and dozens wounded as blasts triggered by. at least 18 killed as bomb attacks rock central baghdad ajenglish. at least 21 killed and dozens wounded as blasts triggered by suicide bomber and explosive ajenglish. 2 blasts in central baghdad kill at least 18 police say afp al jazeera. dozens also injured as two bombs explode at busy market in central al sinak neighbourhood. at least 21 killed and dozens wounded in baghdad after a suicide bombing

['two bombs explode at a busy market in central baghdad causing at least 27 people to be killed and 57 others to be wounded the explosions occurred near car spare parts shops and in the market the explosions were triggered by a suicide bomber and an explosive the incident has been widely reported in the news with some expressing shock

In [30]:
i = 78
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

delta bans disruptive trump supporter for life after pro trumpantihillary tirade video goes viral. bravo delta delta air lines bans disruptive donald trump supporter for life. loud rude and disrespectful was it actually trump they banned delta bans disruptive trump supporter for life. delta air lines bans disruptive donald trump supporter for life thankyou delta. delta air lines bans disruptive donald trump supporter for life bravo delta airlines. delta air lines bans disruptive donald trump supporter for life new york times. well done delta delta air lines bans disruptive donald trump supporter for life. fair treatment for all delta air lines bans disruptive donald trump supporter for life

['delta air lines has banned a disruptive donald trump supporter from flying for life after a video of the supporter expressing a protrump antihillary tirade went viral the incident has been widely reported and has been widely criticized with some expressing support for the president and others exp

In [32]:
i = 150
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

swap rail car for bus it is a holocaust shameful syrians in rebel areas doom or the green bus. must ready by abarnardnyt and hwaida_saad stark choice for syrians in rebel areas doom or the green bus. latest news update from luisa florez stark choice for syrians in rebel areas doom or the green bus. imagine facing this choice with your family stark choice for syrians in rebel areas doom or the green bus. latest news update from luisa florez stark choice for syrians in rebel areas doom or the green bus. swap rail car for bus it is a holocaust shameful syrians in rebel areas doom or the green bus. stark choice for syrians in rebel areas doom or the green bus check out via nyt the new york times. sad piece about syria though beautifully written stark choice 4syrians in rebel areas doom or the green bus

['the paragraphs report on a stark choice for syrians in rebel areas facing doom or the green bus the focus is on the piece by luisa florez and her commentary on the issue with some express

In [33]:
i = 257
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

huge victory standingrock water safety nativeamerican dakotaaccesspipeline army corps not grant easement. army corps denies easement for dakota access pipeline crossing. army corps denies easement for dakotaaccesspipeline crossing abc news. the department of the army will not approve an easement that would have allowed the proposed dakota access pipeline to cross under lake oahe in north dakota the armys assistant secretary for civil works announced today in a statementthe army claimed in a statement that assistant secretary for. the department of the army will not approve an easement that would have allowed the proposed dakota access pipeline to cross under lake oahe in north dakota the armys assistant secretary for civil works announced today in a statementthe army claimed in a statement that assistant secretary for. army corps denies easement for dakota access pipeline crossing. one small step army corps denies easement for dakota access pipeline crossing. army corps denies easement

In [34]:
# Never Seen Before
i = 300
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

authorities found a sixth survivor the man was found under the fuselage officials forcachapecoense ht. colombia plane crash 75 dead some survivors. 6 possible survivors in colombia plane accident officials say. 6 possible survivors in colombia plane accident officials say. colombia plane crash 6th survivor found cnn. cnnbrk authorities find a sixth survivor of a plane crash in colombia the man was found under the fuselage of. colombia plane crash 6th survivor found estimated 75 killed. black boxes from plane that crashed in colombia have been found according to civil aviation authority

['the paragraphs report on a plane crash in colombia that left 75 people dead and six possible survivors with one man found under the fuselage the article notes that black boxes from the plane that crashed in colombia have been found and the number of survivors is estimated at around six the article notes that the man was found under the fuselage and that the crash was a tragic event with some expressin

After lr finetune, discriminator correction

In [34]:
i = 1
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

excellent executive decision trump wants to cancel air force one order from boeing. build it in mexico it will be cheaper that way trump wants to cancel air force one order from boeing. when did republicans become so fond of regulation trump wants 2cancel air force one order from boeing. trump lies again says jet costs more than 4 billion amp wants to cancel air force one order from boeing. trump lies again says jet costs more than 4 billion amp wants to cancel air force one order from boeing. presidentelect realdonaldtrump wants to cancel a government order for a new air force one. amazing leadership and not even sworn in trump wants to cancel air force one order from boeing. trump wants to cancel air force one order from boeing what about the money for security at trump tower

['the paragraphs report on presidentelect trumps decision to cancel an air force one order from boeing the focus is on the executive decision and the potential impact on security at trump tower the focus is on 

In [35]:
i = 25
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

remarkable photos from nytimes the year in pictures 2016. this is excellent the year in pictures 2016. a year to be confounded shocked nytimes year in pictures 2016. some extraordinary photos here the nytimes year in pictures 2016. the year in pictures 2016 new york times. a treasure here gt gt the year in pictures 2016. a riveting curation the year in pictures 2016. what a year 2016 even tragedy can be captured beautifully the year in pictures 2016

['the new york times has published a year in pictures 2016 with some remarkable photos from the coverage including some from the new york times and others']

the new york times the year in pictures 2016 is a collection of remarkable and extraordinary photographs that capture the tragedies and triumphs of the year many people found the curation to be a riveting and beautiful depiction of a year that left people confounded and shocked the photos are described as a treasure and a great piece and they demonstrate that even in tragedy beauty ca

In [36]:
i = 4
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

the new york times how to pounce on best credit card offers. how to pounce on best credit card offers before banks pull them. how to search for the best credit card deals before the disappear. how to pounce on best credit card offers last chance for freemoney just pick it up. 1500 in your pocket ronlieber explains how to pounce on the best credit card offers. how to pounce on best credit card offers por ron lieber en nyt. how to pounce on best credit card offers by ron lieber the new york times. new york times most viewed stories how to pounce on best credit card offers

['the article discusses how to search for the best credit card offers before banks pull them out the article has been widely viewed and has garnered attention from readers and other sources']

the article discusses how to take advantage of credit card offers before they expire and how to search for the best deals ron lieber explains how to earn free money by pouncing on credit card offers and the article is one of the 

In [37]:
i = 9
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

38 dead and 155 wounded in twin bomb attack near istanbul stadium who is really behind this atta. 29 dead and 166 wounded in twin bomb attack near istanbul stadium. terror in istanbul 29 dead 166 wounded in twin bomb attack in turkey. thoughts and prayers to the victims 38 dead and 155 wounded in twin bomb attack near istanbul stadium foxnews. 15 dead 69 wounded in twin bomb attack in istanbul. thoughts and prayers to the victims 38 dead and 155 wounded in twin bomb attack near istanbul stadium foxnews. 38 dead and 155 wounded in twin bomb attack near istanbul stadium what goes around comes around. 38 dead and 155 wounded in twin bomb attack near istanbul stadium what goes around comes around

['the paragraphs report on the 38 dead and 155 wounded in a twin bomb attack near istanbul stadium the focus is on the terror attacks perpetrators and their thoughts and prayers']

a twin bomb attack near an istanbul stadium has resulted in 38 dead and 155 wounded the attack has been widely conde

In [38]:
i = 15
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])
print(get_discriminator(train_dataset[i]))

at least 27 people killed and 57 others wounded when blasts went off near car spare parts shops in al sinak. two blasts in central baghdad kill at least 28. double bomb attack hits al sinak market at least 21 killed and dozens wounded as blasts triggered by. at least 18 killed as bomb attacks rock central baghdad ajenglish. at least 21 killed and dozens wounded as blasts triggered by suicide bomber and explosive ajenglish. 2 blasts in central baghdad kill at least 18 police say afp al jazeera. dozens also injured as two bombs explode at busy market in central al sinak neighbourhood. at least 21 killed and dozens wounded in baghdad after a suicide bombing

['two bombs explode in central baghdad near car spare parts shops killing at least 28 people and injuring 57 others the explosions occurred near a busy market in al sinak and were triggered by a suicide bomber and explosive the blasts occurred in the central market and were triggered by the suicide bomber and explosive ajenglish the b

In [41]:
i = 78
print(test_dataset[i]['input_text'])
print()
print(get_generated_summary(test_dataset[i]))
print()
print(test_dataset[i]['target_text'])
print(get_discriminator(test_dataset[i]))

delta banned racist trumpkin fliyer for life. delta banned racist trumpkin fliyer for life. bravo delta delta air lines bans disruptive donald trump supporter for life. banned and refunded others delta air lines bans disruptive donald trump supporter for life

['delta air lines has banned and refunded others a racist trumpkin fliyer for life and has banned and refunded others the airline has also banned and refunded others the airline has also been criticized for its actions and its handling of the situation']

delta air lines has banned a disruptive donald trump supporter from flying for life after a video of the individuals protrump and antihillary tirade went viral on social media the incident has gained attention in the media with many expressing support for deltas decision and criticizing the individuals behavior on the flight some have also raised concerns about the impact of political polarization and the role of social media in shaping public opinion and discourse
(tensor([[9.8

In [44]:
i = 150
print(test_dataset[i]['input_text'])
print()
print(get_generated_summary(test_dataset[i]))
print()
print(test_dataset[i]['target_text'])
print(get_discriminator(test_dataset[i]))

heartbreaking empathetic reporting wo an agenda stark choice for syrians in rebel areas doom or the green bus. great story from abarnardnyt et al stark choice for syrians in rebel areas doom or the green bus. heartbreaking empathetic reporting wo an agenda stark choice for syrians in rebel areas doom or the green bus. imagine facing this choice with your family stark choice for syrians in rebel areas doom or the green bus

['the paragraphs report on a heartbreaking and empathetic story about a political agenda that has left many syrians facing a choice between doom and the green bus the focus is on the story and the implications for syrians in rebel areas']

the article discusses the stark choice faced by syrians in rebel areas between staying and facing possible doom or taking the green bus and leaving the situation is described as a holocaust and the emotional impact of this decision on families is highlighted the reporting is empathetic and heartbreaking with no particular agenda an

In [45]:
i = 257
print(test_dataset[i]['input_text'])
print()
print(get_generated_summary(test_dataset[i]))
print()
print(test_dataset[i]['target_text'])
print(get_discriminator(test_dataset[i]))

army corps says no to dakota access pipeline. show them how it is done army corps denies easement for dakota access pipeline crossing. army corps denies easement for dakota access pipeline crossing amp alternate routes to be examined. one small step army corps denies easement for dakota access pipeline crossing

['the paragraphs report on the army corps denying an easement for the dakota access pipeline crossing and discuss alternate routes to be examined the focus is on the army corps decision to deny the crossing and the importance of showing them how it is done']

the paragraphs report on the decision by the us army corps of engineers to deny an easement for the dakota access pipeline to cross under lake oahe in north dakota which is considered a huge victory for standing rock sioux tribe and activists concerned about water safety and native american rights some articles mention the armys announcement and the potential implications of the decision while others express support for th

In [43]:
# Never Seen Before Prompt
i = 300
print(test_dataset[i]['input_text'])
print()
print(get_generated_summary(test_dataset[i]))
print()
print(test_dataset[i]['target_text'])
print(get_discriminator(test_dataset[i]))

authorities say 76 people are confirmed dead after a plane carrying a soccer team crashed in colombia. brazilian soccer team plane crash 6th survivor found under fuselage. authorities find a sixth survivor from plane crash in colombia the man was found under the fuselage official says. 6 possible survivors in colombia plane accident officials say

['a plane carrying a brazilian soccer team crashed in colombia killing 76 people the man was found under the fuselage and is the sixth person to survive the crash officials say there are no survivors of the crash the crash has been reported by several news outlets including cnn and cnn']

authorities found a sixth survivor from the colombia plane crash in which 75 people were killed the man was found under the fuselage and there were initially six possible survivors according to officials the black boxes from the plane have been found and the civil aviation authority confirms that 76 people are dead the survivor was a member of the brazilian 

Initial model with no discriminator

In [42]:
i = 1
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])

excellent executive decision trump wants to cancel air force one order from boeing. build it in mexico it will be cheaper that way trump wants to cancel air force one order from boeing. when did republicans become so fond of regulation trump wants 2cancel air force one order from boeing. trump lies again says jet costs more than 4 billion amp wants to cancel air force one order from boeing. trump lies again says jet costs more than 4 billion amp wants to cancel air force one order from boeing. presidentelect realdonaldtrump wants to cancel a government order for a new air force one. amazing leadership and not even sworn in trump wants to cancel air force one order from boeing. trump wants to cancel air force one order from boeing what about the money for security at trump tower

['julian zelizer: trump lies again says jet costs more than 4 billion amp wants to cancel order. zelizer: amazing leadership and not even sworn in trump wants to cancel order from boeing. he says trump lies aga

In [77]:
i = 25
print(test_dataset[i]['input_text'])
print()
print(get_generated_summary(test_dataset[i]))
print()
print(test_dataset[i]['target_text'])

this is excellent the year in pictures 2016. the year in pictures 2016 new york times. the nyt the year in pictures 2016. such a great piece the year in pictures 2016

["aaron carroll: this is excellent the year in pictures 2016. carroll: it's a great piece, and the year in pictures 2016 is a great year. he says the year in pictures 2016 is a great year, and the new york times is great. carroll: the year in pictures 2016 is a great year, and the year in pictures 2016 is a great year."]

the new york times the year in pictures 2016 is a collection of remarkable and extraordinary photographs that capture the tragedies and triumphs of the year many people found the curation to be a riveting and beautiful depiction of a year that left people confounded and shocked the photos are described as a treasure and a great piece and they demonstrate that even in tragedy beauty can be found


In [49]:
i = 1
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])

excellent executive decision trump wants to cancel air force one order from boeing. build it in mexico it will be cheaper that way trump wants to cancel air force one order from boeing. when did republicans become so fond of regulation trump wants 2cancel air force one order from boeing. trump lies again says jet costs more than 4 billion amp wants to cancel air force one order from boeing. trump lies again says jet costs more than 4 billion amp wants to cancel air force one order from boeing. presidentelect realdonaldtrump wants to cancel a government order for a new air force one. amazing leadership and not even sworn in trump wants to cancel air force one order from boeing. trump wants to cancel air force one order from boeing what about the money for security at trump tower

['julian zelizer: trump lies again says jet costs more than 4 billion amp wants to cancel order. zelizer: amazing leadership and not even sworn in trump wants to cancel order from boeing. he says trump lies aga

In [51]:
i = 4
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])

the new york times how to pounce on best credit card offers. how to pounce on best credit card offers before banks pull them. how to search for the best credit card deals before the disappear. how to pounce on best credit card offers last chance for freemoney just pick it up. 1500 in your pocket ronlieber explains how to pounce on the best credit card offers. how to pounce on best credit card offers por ron lieber en nyt. how to pounce on best credit card offers by ron lieber the new york times. new york times most viewed stories how to pounce on best credit card offers

["the new york times explains how to pounce on the best credit card offers. ron lieber explains how to search for the best credit card deals before they disappear. lieber: i'm a pouncer, and i'm not a pouncer."]

the article discusses how to take advantage of credit card offers before they expire and how to search for the best deals ron lieber explains how to earn free money by pouncing on credit card offers and the ar

In [57]:
i = 9
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])

38 dead and 155 wounded in twin bomb attack near istanbul stadium who is really behind this atta. 29 dead and 166 wounded in twin bomb attack near istanbul stadium. terror in istanbul 29 dead 166 wounded in twin bomb attack in turkey. thoughts and prayers to the victims 38 dead and 155 wounded in twin bomb attack near istanbul stadium foxnews. 15 dead 69 wounded in twin bomb attack in istanbul. thoughts and prayers to the victims 38 dead and 155 wounded in twin bomb attack near istanbul stadium foxnews. 38 dead and 155 wounded in twin bomb attack near istanbul stadium what goes around comes around. 38 dead and 155 wounded in twin bomb attack near istanbul stadium what goes around comes around

['turkey: 38 dead, 155 wounded in twin bomb attack near istanbul stadium. foxnews: 166 wounded in twin bomb attack near istanbul stadium. foxnews: russian prime minister vladimir putin vladimir putin vladimir putin.']

a twin bomb attack near an istanbul stadium has resulted in 38 dead and 155 wo

In [66]:
i = 15
print(train_dataset[i]['input_text'])
print()
print(get_generated_summary(train_dataset[i]))
print()
print(train_dataset[i]['target_text'])

at least 27 people killed and 57 others wounded when blasts went off near car spare parts shops in al sinak. two blasts in central baghdad kill at least 28. double bomb attack hits al sinak market at least 21 killed and dozens wounded as blasts triggered by. at least 18 killed as bomb attacks rock central baghdad ajenglish. at least 21 killed and dozens wounded as blasts triggered by suicide bomber and explosive ajenglish. 2 blasts in central baghdad kill at least 18 police say afp al jazeera. dozens also injured as two bombs explode at busy market in central al sinak neighbourhood. at least 21 killed and dozens wounded in baghdad after a suicide bombing

['double bomb attack hits al sinak market killing at least 28 people. explosions triggered by suicide bomber and explosive ajenglish. at least 21 killed and dozens wounded in baghdad after a suicide bombing.']

two blasts in central baghdad specifically in the al sinak market resulted in the deaths of at least 27 people and wounded 57